In [1]:
import os 
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/usr/hdp/current/spark2-client"
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-10_2.12:3.4.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.4.0,org.postgresql:postgresql:42.6.0 pyspark-shell'

import findspark
findspark.init()
findspark.add_packages([
    "org.apache.spark:spark-streaming-kafka-0-10_2.12:3.4.0",
    "org.apache.spark:spark-sql-kafka-0-10_2.12:3.4.0",
    "org.postgresql:postgresql:42.6.0"
])

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
SOCRATA_APP_TOKEN=os.getenv("SOCRATA_APP_TOKEN")
KAFKA_BOOTSTRAP_SERVER=os.getenv("KAFKA_BOOTSTRAP_SERVER")
KAFKA_TOPIC=os.getenv("KAFKA_TOPIC")

In [3]:
print(KAFKA_BOOTSTRAP_SERVER)
print(KAFKA_TOPIC)

10.67.165.99:9092
chicago_congestion_2


https://towardsdatascience.com/a-fast-look-at-spark-structured-streaming-kafka-f0ff64107325

In [4]:
from pyspark.sql import SparkSession 

spark = SparkSession.builder.master('local[*]').appName("capstone_streaming").getOrCreate()

:: loading settings :: url = jar:file:/home/minhpc/int3229e/.venv/lib/python3.7/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/minhpc/.ivy2/cache
The jars for the packages stored in: /home/minhpc/.ivy2/jars
org.apache.spark#spark-streaming-kafka-0-10_2.12 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-9ddcc1eb-a3f7-40b5-917c-f0c6c3028fd0;1.0
	confs: [default]
	found org.apache.spark#spark-streaming-kafka-0-10_2.12;3.4.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.4.0 in central
	found org.apache.kafka#kafka-clients;3.3.2 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.9.1 in central
	found org.slf4j#slf4j-api;2.0.6 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in cent

In [5]:
df = spark.readStream.format("kafka") \
    .option("kafka.bootstrap.servers", KAFKA_BOOTSTRAP_SERVER) \
    .option("subscribe", KAFKA_TOPIC) \
    .option("startingOffsets", "earliest") \
    .load()

In [6]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, TimestampType

streaming_schema = StructType(
    [
        StructField("segmentid", StringType(), True),
        StructField("street", StringType(), True),
        StructField("_direction", StringType(), True),
        StructField("_fromst", StringType(), True),
        StructField("_tost", StringType(), True),
        StructField("_length", StringType(), True),
        StructField("_strheading", StringType(), True),
        StructField("_comments", StringType(), True),
        StructField("start_lon", DoubleType(), True),
        StructField("_lif_lat", DoubleType(), True),
        StructField("_lit_lon", DoubleType(), True),
        StructField("_lit_lat", DoubleType(), True),
        StructField("_traffic", StringType(), True),
        StructField("_last_updt", TimestampType(), True)
    ]
)

In [18]:
from pyspark.sql import DataFrame
from pyspark.sql.functions import from_json, col, lit

def transform_to_f_congestion(df: DataFrame) -> DataFrame:
    df = df.withColumn("value", from_json(col("value"), streaming_schema)).select("value.*")

    df = df.selectExpr(
        "*",
        "date_format(_last_updt + INTERVAL 5 HOURS, 'yyyyMMddHHmm') AS gmtts_str",
        "CAST(format_string('%04d', CAST(segmentid AS int)) AS string) AS segment_padded"
    )
    df = df.selectExpr(
        "CONCAT_WS('-', segment_padded, gmtts_str) AS record_id",
        "CAST(segmentid AS INT) AS segment_id",
        "_last_updt AS timestamp",
        "gmtts_str",
        "CAST(_traffic AS INT) AS speed",
        "CAST(NULL AS INT) as bus_count",
        "CAST(NULL AS INT) as message_count",
        "CAST(date_format(_last_updt, 'yyyyMMdd') AS INT) AS date_key",
        "CAST(date_format(_last_updt, 'HH') AS INT) AS hour_key",
    )

    return df

def append_to_psql(df: DataFrame) -> DataFrame:
    prop = {
        'url': os.getenv("DEST_PSQL_URL"),
        "driver": 'org.postgresql.Driver',
        'user': os.getenv("DEST_PSQL_USER"),
        'password': os.getenv("DEST_PSQL_PASS")
    }
    
    df.write.jdbc(url=prop['url'], table='f_congestion', mode='append', properties=prop)

def save_to_file(df: DataFrame, batch_id: int):
    df \
        .write.format('csv').option('path', 'stream.csv').option('header', 'true').mode('overwrite').save()

In [19]:
df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)") \
    .writeStream \
    .outputMode("append") \
    .foreachBatch(lambda df, batch_id: append_to_psql(transform_to_f_congestion(df))) \
    .start() \
    .awaitTermination()

23/05/27 06:45:03 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-56f2456a-3391-491f-9177-4a543954b1fb. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/05/27 06:45:03 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
23/05/27 06:45:03 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.
ERROR:root:KeyboardInterrupt while sending command.                             
Traceback (most recent call last):
  File "/home/minhpc/int3229e/.venv/lib/python3.7/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(comma

KeyboardInterrupt: 

In [33]:
df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)") \
    .writeStream \
    .format("console") \
    .outputMode("append") \
    .start() \
    .awaitTermination()

23/05/27 06:27:26 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-50caebd7-a239-4c02-934d-f99796e75841. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/05/27 06:27:26 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
23/05/27 06:27:26 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


-------------------------------------------
Batch: 0
-------------------------------------------
+----+--------------------+
| key|               value|
+----+--------------------+
|null|{"segmentid": "11...|
|null|{"segmentid": "11...|
|null|{"segmentid": "11...|
|null|{"segmentid": "11...|
|null|{"segmentid": "11...|
|null|{"segmentid": "11...|
|null|{"segmentid": "12...|
|null|{"segmentid": "12...|
|null|{"segmentid": "12...|
|null|{"segmentid": "12...|
|null|{"segmentid": "12...|
|null|{"segmentid": "12...|
|null|{"segmentid": "12...|
|null|{"segmentid": "12...|
|null|{"segmentid": "12...|
|null|{"segmentid": "12...|
|null|{"segmentid": "13...|
|null|{"segmentid": "13...|
|null|{"segmentid": "13...|
|null|{"segmentid": "13...|
+----+--------------------+
only showing top 20 rows



ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/home/minhpc/int3229e/.venv/lib/python3.7/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/minhpc/int3229e/.venv/lib/python3.7/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.7/socket.py", line 589, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 